# WitnessChain Interactive Tutorial

## Understanding Login and Challenge Creation in the WitnessChain Ecosystem

This interactive notebook will guide you through the process of:
1. **Setting up your environment** for WitnessChain integration
2. **Logging into the WitnessChain** ecosystem
3. **Creating and managing challenges** for proof verification

### What is WitnessChain?

WitnessChain is a decentralized proof verification system that allows you to:
- Create challenges for various types of proofs (Proof of Location, Proof of Work, etc.)
- Verify proofs through a network of challengers
- Manage campaigns and API keys for automated operations

Let's start by understanding the core concepts and then dive into practical examples.


## 1. Environment Setup

Before we can interact with WitnessChain, we need to set up our environment properly. This includes:

### Prerequisites
- Python 3.7+
- An Ethereum private key (for authentication)
- Required Python packages

### Step 1: Install Required Packages


In [ ]:
# Install required packages
# Run this cell if you haven't installed the packages yet
# !pip install requests eth-account

# Import necessary libraries
import os
import sys
import json
import requests
from eth_account.messages import encode_defunct
from eth_account import Account

# Import our WitnessChain API class
from witnesschain import api

print("✅ All required packages imported successfully!")
print("📦 Available packages:")
print(f"   - requests: {requests.__version__}")
print(f"   - eth_account: {Account.__version__}")


### Step 2: Set Up Your Private Key

**⚠️ IMPORTANT SECURITY NOTE**: Never commit your private key to version control or share it publicly. Always use environment variables or secure key management.

For this tutorial, you have a few options:

1. **Set environment variable** (Recommended for production):
   ```bash
   export PRIVATE_KEY="your_private_key_here"
   ```

2. **Use a test private key** (For learning purposes only):
   ```python
   # WARNING: This is a test key - DO NOT use in production!
   test_private_key = "0x1234567890abcdef1234567890abcdef1234567890abcdef1234567890abcdef"
   ```

Let's check if your private key is properly set up:


In [ ]:
# Check if private key is set in environment
if "PRIVATE_KEY" in os.environ:
    private_key = os.environ["PRIVATE_KEY"]
    print("✅ Private key found in environment variables")
    
    # Verify the private key and get the address
    try:
        account = Account.from_key(private_key)
        address = account.address
        print(f"📍 Your Ethereum address: {address}")
        print(f"🔑 Private key (first 10 chars): {private_key[:10]}...")
    except Exception as e:
        print(f"❌ Invalid private key: {e}")
        sys.exit(1)
else:
    print("⚠️  PRIVATE_KEY environment variable not found")
    print("📝 Please set your private key using one of these methods:")
    print("   1. Set environment variable: export PRIVATE_KEY='your_key_here'")
    print("   2. Or modify the code below to use a test key")
    
    # Uncomment the lines below if you want to use a test key for learning
    # WARNING: Only use test keys for learning purposes!
    # test_private_key = "0x1234567890abcdef1234567890abcdef1234567890abcdef1234567890abcdef"
    # os.environ["PRIVATE_KEY"] = test_private_key
    # print("🧪 Using test private key for demonstration")


## 2. Understanding the Login Process

The WitnessChain login process is a two-step authentication mechanism that ensures secure access to the platform:

### Login Flow Overview

1. **Pre-login**: Send your public key and role information
2. **Receive challenge**: Get a message to sign
3. **Sign message**: Use your private key to sign the challenge
4. **Login**: Submit the signature to complete authentication

### Available Networks and Proof Types

**Networks:**
- `testnet`: For testing and development
- `mainnet`: For production use

**Proof Types:**
- `pol`: Proof of Location
- `pow`: Proof of Work
- `pos`: Proof of Stake
- And more...

Let's initialize a WitnessChain client and explore the login process:


In [ ]:
# Initialize WitnessChain client
# For this tutorial, we'll use testnet for safety
network = "testnet"  # Change to "mainnet" for production
proof_type = "pol"   # Proof of Location

print(f"🌐 Initializing WitnessChain client...")
print(f"   Network: {network}")
print(f"   Proof Type: {proof_type}")

# Create the API client
try:
    wtns_client = api(network, proof_type)
    print("✅ WitnessChain client initialized successfully!")
    print(f"📍 Your address: {wtns_client.address}")
    print(f"🔗 Base URL: {wtns_client.BASE_URL}")
except Exception as e:
    print(f"❌ Failed to initialize client: {e}")
    print("💡 Make sure your PRIVATE_KEY environment variable is set correctly")


### Step-by-Step Login Process

Now let's break down the login process and see what happens behind the scenes:


In [ ]:
# Step 1: Pre-login - Send your public key and role information
print("🔐 Step 1: Pre-login")
print("   Sending public key and role information...")

# This is what happens internally in the login() method
pre_login_data = {
    "role": "payer",
    "keyType": "ethereum", 
    "publicKey": wtns_client.address,
    "clientVersion": "9999999999",
    "walletPublicKey": {
        "ethereum": wtns_client.address
    }
}

print(f"   📤 Pre-login data: {json.dumps(pre_login_data, indent=2)}")

# Make the pre-login request
try:
    pre_login_response = wtns_client.do_post(
        "pre-login",
        json.dumps(pre_login_data)
    )
    
    if pre_login_response:
        print("✅ Pre-login successful!")
        print(f"   📨 Received message to sign: {pre_login_response.get('message', 'N/A')}")
        
        # Step 2: Sign the message
        print("\n✍️  Step 2: Signing the message")
        message_to_sign = pre_login_response["message"]
        signature = wtns_client.sign(message_to_sign)
        print(f"   📝 Message: {message_to_sign}")
        print(f"   🔏 Signature: {signature}")
        
        # Step 3: Complete login with signature
        print("\n🚪 Step 3: Completing login")
        login_data = {"signature": signature}
        print(f"   📤 Login data: {json.dumps(login_data, indent=2)}")
        
        login_response = wtns_client.do_post(
            "login", 
            json.dumps(login_data)
        )
        
        if login_response:
            print("✅ Login completed successfully!")
            print(f"   🎉 Login response: {json.dumps(login_response, indent=2)}")
        else:
            print("❌ Login failed")
    else:
        print("❌ Pre-login failed")
        
except Exception as e:
    print(f"❌ Error during login process: {e}")
    print("💡 This might be due to network issues or invalid credentials")


In [ ]:
# Simplified login using the built-in method
print("🔄 Simplified Login Process")
print("   Using the built-in login() method...")

try:
    # This is the simple way to login
    login_result = wtns_client.login()
    
    if login_result:
        print("✅ Login successful using built-in method!")
        print(f"   🎉 Result: {json.dumps(login_result, indent=2)}")
    else:
        print("❌ Login failed")
        
except Exception as e:
    print(f"❌ Login error: {e}")
    print("💡 Make sure you have a valid private key and network connection")


## 3. Creating and Managing Challenges

Now that we're logged in, let's explore how to create and manage challenges in the WitnessChain ecosystem.

### What is a Challenge?

A challenge in WitnessChain is a request to verify a specific proof. When you create a challenge, you're asking the network to verify whether a particular claim (like proof of location) is valid.

### Challenge Components

- **Prover**: The entity making the claim (e.g., a watchtower address)
- **Challenge Type**: The type of proof to verify (e.g., "pol" for Proof of Location)
- **Number of Challengers**: How many validators should participate in the verification
- **API Key** (optional): For automated operations

Let's start by checking your account balance and then create a challenge:


In [ ]:
# Check your account balance
print("💰 Checking Account Balance")
print("   Getting your current balance...")

try:
    balance = wtns_client.get_balance()
    if balance:
        print("✅ Balance retrieved successfully!")
        print(f"   💵 Balance: {json.dumps(balance, indent=2)}")
    else:
        print("❌ Failed to retrieve balance")
except Exception as e:
    print(f"❌ Error getting balance: {e}")
    print("💡 You might need to login first or check your network connection")


### Creating a Challenge

Now let's create a challenge. For this example, we'll use a sample watchtower address and create a Proof of Location challenge:


In [ ]:
# Example watchtower address (this would be a real watchtower in production)
watchtower_address = "IPv4/0x939744500de04b4e2d5d68d233617a5ac6968aa0"

# Challenge parameters
challenge_type = "pol"  # Proof of Location
num_challengers = 1     # Number of validators to participate

print("🎯 Creating a Challenge")
print(f"   📍 Watchtower Address: {watchtower_address}")
print(f"   🔍 Challenge Type: {challenge_type}")
print(f"   👥 Number of Challengers: {num_challengers}")

try:
    # Create the challenge
    challenge_response = wtns_client.request_challenge(
        prover=watchtower_address,
        num_challengers=num_challengers,
        challenge_type=challenge_type
    )
    
    if challenge_response:
        print("✅ Challenge created successfully!")
        print(f"   🎉 Challenge Response: {json.dumps(challenge_response, indent=2)}")
        
        # Extract challenge ID for future reference
        challenge_id = challenge_response.get('challenge_id')
        if challenge_id:
            print(f"   🆔 Challenge ID: {challenge_id}")
            print("   💡 Save this ID to check the challenge status later!")
    else:
        print("❌ Failed to create challenge")
        
except Exception as e:
    print(f"❌ Error creating challenge: {e}")
    print("💡 Make sure you have sufficient balance and valid parameters")


### Checking Challenge Status

Once you've created a challenge, you can monitor its progress by checking the status:


In [ ]:
# Check challenge status (replace with actual challenge ID from previous step)
# challenge_id = "your_challenge_id_here"  # Replace with actual ID

# For demonstration, let's show how to check status
print("📊 Checking Challenge Status")
print("   To check a challenge status, you need the challenge ID from the creation response")

# Example of how to check status (uncomment and modify when you have a real challenge ID)
# try:
#     status_response = wtns_client.challenge_status(challenge_id)
#     if status_response:
#         print("✅ Status retrieved successfully!")
#         print(f"   📈 Status: {json.dumps(status_response, indent=2)}")
#     else:
#         print("❌ Failed to retrieve status")
# except Exception as e:
#     print(f"❌ Error checking status: {e}")

print("💡 To check a real challenge status:")
print("   1. Create a challenge first (run the previous cell)")
print("   2. Copy the challenge_id from the response")
print("   3. Uncomment and modify the code above with your challenge ID")


## 4. Advanced Features and Examples

Let's explore some advanced features of the WitnessChain API:

### API Key Management

For automated operations, you can create and manage API keys:


In [ ]:
# Create an API key for automated operations
print("🔑 Creating API Key")
print("   Creating an API key for automated challenge requests...")

try:
    # Create API key
    api_key_name = "tutorial_api_key"
    valid_till_days = 30  # Valid for 30 days
    
    api_key_response = wtns_client.create_apikey(
        name=api_key_name,
        valid_till_days=valid_till_days
    )
    
    if api_key_response:
        print("✅ API key created successfully!")
        print(f"   🔑 API Key Response: {json.dumps(api_key_response, indent=2)}")
        
        # Extract the API key for future use
        api_key = api_key_response.get('api_key')
        if api_key:
            print(f"   🎫 Your API Key: {api_key}")
            print("   ⚠️  Keep this API key secure and don't share it publicly!")
    else:
        print("❌ Failed to create API key")
        
except Exception as e:
    print(f"❌ Error creating API key: {e}")
    print("💡 Make sure you're logged in and have the necessary permissions")


In [ ]:
# List all your API keys
print("📋 Listing All API Keys")
print("   Getting all your API keys...")

try:
    all_api_keys = wtns_client.get_all_apikeys()
    if all_api_keys:
        print("✅ API keys retrieved successfully!")
        print(f"   📝 All API Keys: {json.dumps(all_api_keys, indent=2)}")
    else:
        print("❌ Failed to retrieve API keys")
except Exception as e:
    print(f"❌ Error retrieving API keys: {e}")


### Creating Challenges with API Keys

Now let's create a challenge using an API key for automated operations:


In [ ]:
# Create a challenge using API key (for automated operations)
print("🤖 Creating Challenge with API Key")
print("   Using API key for automated challenge creation...")

# Example API key (replace with your actual API key from previous step)
# api_key = "your_api_key_here"  # Replace with actual API key

# For demonstration purposes, we'll show the structure
print("💡 To create a challenge with API key:")
print("   1. Get your API key from the previous step")
print("   2. Uncomment and modify the code below")

# Example code (uncomment and modify when you have a real API key)
# try:
#     challenge_with_api = wtns_client.request_challenge(
#         prover=watchtower_address,
#         num_challengers=2,  # More challengers for better verification
#         challenge_type="pol",
#         apikey=api_key  # Use API key for authentication
#     )
#     
#     if challenge_with_api:
#         print("✅ Challenge created with API key!")
#         print(f"   🎉 Response: {json.dumps(challenge_with_api, indent=2)}")
#     else:
#         print("❌ Failed to create challenge with API key")
#         
# except Exception as e:
#     print(f"❌ Error creating challenge with API key: {e}")

print("\n🔍 Key differences when using API keys:")
print("   - No need to login for each request")
print("   - Better for automated/scripted operations")
print("   - More secure for production environments")


### Campaign Management

WitnessChain also supports campaign management for organizing multiple challenges:


In [ ]:
# Get available campaigns
print("📊 Getting Available Campaigns")
print("   Retrieving all campaigns...")

try:
    campaigns = wtns_client.get_campaigns()
    if campaigns:
        print("✅ Campaigns retrieved successfully!")
        print(f"   📋 Campaigns: {json.dumps(campaigns, indent=2)}")
    else:
        print("❌ Failed to retrieve campaigns")
except Exception as e:
    print(f"❌ Error retrieving campaigns: {e}")

print("\n💡 Campaigns are used to organize multiple challenges")
print("   - Group related challenges together")
print("   - Set common parameters for multiple challenges")
print("   - Track progress across multiple verifications")


## 5. Practical Examples and Use Cases

Let's explore some real-world scenarios where you might use WitnessChain:

### Example 1: IoT Device Location Verification

Imagine you have IoT devices that need to prove their location:


In [ ]:
# Example: IoT Device Location Verification
print("🏠 IoT Device Location Verification Example")
print("   Scenario: Verify that an IoT device is actually at a specific location")

# Simulate multiple IoT devices
iot_devices = [
    {
        "device_id": "sensor_001",
        "location": "New York, NY",
        "watchtower_address": "IPv4/0x939744500de04b4e2d5d68d233617a5ac6968aa0"
    },
    {
        "device_id": "sensor_002", 
        "location": "San Francisco, CA",
        "watchtower_address": "IPv4/0x1234567890abcdef1234567890abcdef12345678"
    }
]

print(f"   📱 Found {len(iot_devices)} IoT devices to verify")

# Create challenges for each device
for device in iot_devices:
    print(f"\n🔍 Verifying device: {device['device_id']}")
    print(f"   📍 Claimed location: {device['location']}")
    print(f"   🏗️  Watchtower: {device['watchtower_address']}")
    
    # In a real scenario, you would create challenges here
    print("   💡 To create challenge:")
    print(f"      challenge_response = wtns_client.request_challenge(")
    print(f"          prover='{device['watchtower_address']}',")
    print(f"          num_challengers=3,  # Multiple challengers for reliability")
    print(f"          challenge_type='pol'")
    print(f"      )")

print("\n🎯 Benefits of this approach:")
print("   - Prevents location spoofing")
print("   - Ensures data integrity")
print("   - Provides cryptographic proof of location")


### Example 2: Automated Monitoring System

Here's how you might set up an automated system to monitor multiple watchtowers:


In [ ]:
# Example: Automated Monitoring System
print("🤖 Automated Monitoring System Example")
print("   Scenario: Continuously monitor multiple watchtowers for location verification")

import time
from datetime import datetime

# Simulate a monitoring system
def monitor_watchtowers(watchtowers, api_key=None):
    """
    Monitor multiple watchtowers and create challenges periodically
    """
    print(f"🔄 Starting monitoring for {len(watchtowers)} watchtowers")
    
    for i, watchtower in enumerate(watchtowers):
        print(f"\n📡 Monitoring watchtower {i+1}: {watchtower['name']}")
        print(f"   📍 Location: {watchtower['location']}")
        print(f"   🏗️  Address: {watchtower['address']}")
        
        # In a real system, you would:
        # 1. Create a challenge
        # 2. Wait for results
        # 3. Log the outcome
        # 4. Repeat after a delay
        
        print("   💡 Automated challenge creation would happen here:")
        print("      - Create challenge with API key")
        print("      - Monitor challenge status")
        print("      - Log results to database")
        print("      - Schedule next verification")

# Example watchtowers to monitor
watchtowers = [
    {
        "name": "NYC Watchtower",
        "location": "New York, NY",
        "address": "IPv4/0x939744500de04b4e2d5d68d233617a5ac6968aa0"
    },
    {
        "name": "SF Watchtower", 
        "location": "San Francisco, CA",
        "address": "IPv4/0x1234567890abcdef1234567890abcdef12345678"
    },
    {
        "name": "London Watchtower",
        "location": "London, UK", 
        "address": "IPv4/0xabcdef1234567890abcdef1234567890abcdef12"
    }
]

# Run the monitoring simulation
monitor_watchtowers(watchtowers)

print("\n🔧 Production Implementation Tips:")
print("   - Use API keys for authentication")
print("   - Implement proper error handling")
print("   - Add logging and monitoring")
print("   - Use database to store results")
print("   - Implement retry logic for failed requests")
print("   - Set up alerts for critical failures")


## 6. Best Practices and Security Considerations

### Security Best Practices

1. **Private Key Management**
   - Never hardcode private keys in your code
   - Use environment variables or secure key management systems
   - Consider using hardware wallets for production

2. **API Key Security**
   - Store API keys securely
   - Rotate API keys regularly
   - Use different API keys for different environments

3. **Network Security**
   - Always use HTTPS endpoints
   - Validate all responses from the API
   - Implement proper error handling

### Error Handling and Resilience

Let's create a robust example with proper error handling:


In [ ]:
# Robust error handling example
def create_challenge_safely(client, prover, num_challengers, challenge_type, max_retries=3):
    """
    Create a challenge with proper error handling and retry logic
    """
    for attempt in range(max_retries):
        try:
            print(f"🔄 Attempt {attempt + 1}/{max_retries}")
            
            # Create the challenge
            response = client.request_challenge(
                prover=prover,
                num_challengers=num_challengers,
                challenge_type=challenge_type
            )
            
            if response:
                print("✅ Challenge created successfully!")
                return response
            else:
                print("❌ Challenge creation failed - no response")
                
        except requests.exceptions.ConnectionError as e:
            print(f"🌐 Connection error: {e}")
            print("   💡 Check your internet connection")
            
        except requests.exceptions.Timeout as e:
            print(f"⏰ Request timeout: {e}")
            print("   💡 The server might be busy, retrying...")
            
        except requests.exceptions.HTTPError as e:
            print(f"🚫 HTTP error: {e}")
            print("   💡 Check your credentials and API endpoint")
            
        except Exception as e:
            print(f"❌ Unexpected error: {e}")
            print("   💡 Check your parameters and try again")
        
        if attempt < max_retries - 1:
            wait_time = 2 ** attempt  # Exponential backoff
            print(f"⏳ Waiting {wait_time} seconds before retry...")
            time.sleep(wait_time)
    
    print("❌ All retry attempts failed")
    return None

# Example usage with error handling
print("🛡️  Robust Challenge Creation Example")
print("   Demonstrating proper error handling and retry logic")

# This would be used in production
# result = create_challenge_safely(
#     client=wtns_client,
#     prover="IPv4/0x939744500de04b4e2d5d68d233617a5ac6968aa0",
#     num_challengers=2,
#     challenge_type="pol"
# )


## 7. Summary and Next Steps

### What We've Learned

In this tutorial, we've covered:

1. **Environment Setup**: How to configure your development environment with proper security
2. **Authentication**: The two-step login process using Ethereum signatures
3. **Challenge Creation**: How to create and manage proof verification challenges
4. **API Key Management**: Setting up automated operations with API keys
5. **Best Practices**: Security considerations and error handling

### Key Takeaways

- **Security First**: Always use environment variables for private keys
- **Error Handling**: Implement robust error handling and retry logic
- **API Keys**: Use API keys for automated operations
- **Monitoring**: Track challenge status and implement proper logging

### Next Steps

1. **Experiment**: Try creating challenges with different parameters
2. **Integrate**: Build this into your own applications
3. **Scale**: Implement automated monitoring systems
4. **Explore**: Check out other proof types and advanced features

### Resources

- **WitnessChain Documentation**: [Official docs]
- **API Reference**: [API endpoints and parameters]
- **Community**: [Discord/Slack channels for support]

### Support

If you encounter any issues:
1. Check your private key and network configuration
2. Verify your account balance
3. Review the error messages and retry logic
4. Reach out to the WitnessChain community for help

Happy coding! 🚀
